In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### vgg_15_BN_64 

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/vgg.yaml \
    train.output_dir /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00 

[2020-06-15 00:00:36] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: vgg
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type:

In [5]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/vgg.yaml \
    test.checkpoint /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00160.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/test_results_0160

[2020-06-15 01:57:10] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00160.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:04<00:00,  8.86it/s]
[2020-06-15 01:57:16] __main__ INFO: Elapsed 4.52
[2020-06-15 01:57:16] __main__ INFO: Loss 0.4304 Accuracy 0.9135


In [6]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/vgg.yaml \
    test.checkpoint /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00100.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/test_results_0100

[2020-06-15 01:57:47] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00100.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:04<00:00,  9.01it/s]
[2020-06-15 01:57:53] __main__ INFO: Elapsed 4.44
[2020-06-15 01:57:53] __main__ INFO: Loss 0.3808 Accuracy 0.9100


In [7]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/vgg.yaml \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00160.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/test_results_0160_CIFAR101

[2020-06-15 01:58:14] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/checkpoint_00160.pth
CIFAR 10.1
100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  6.69it/s]
[2020-06-15 01:58:16] __main__ INFO: Elapsed 1.20
[2020-06-15 01:58:16] __main__ INFO: Loss 0.9958 Accuracy 0.8130


In [8]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['vgg_15_BN_64', 'vgg_15_BN_64', 'vgg_15_BN_64'],
           'Testset': ['cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 160, 160],
           'Loss': [0.3808, 0.4304, 0.9958],
           'Accuracy': [0.9100, 0.9135, 0.8130],
           'Original_Accuracy': [93.0, 93.0, 84.9],
           'Original_CI': [(92.5, 93.5), (92.5, 93.5), (83.2, 86.4)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])

df.to_csv('/home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,vgg_15_BN_64,cifar10,100,0.3808,0.9100,93.0,"(92.5, 93.5)"
1,vgg_15_BN_64,cifar10,160,0.4304,0.9135,93.0,"(92.5, 93.5)"
2,vgg_15_BN_64,cifar10.1,160,0.9958,0.8130,84.9,"(83.2, 86.4)"


In [9]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/vgg_15_BN_64/exp00/test_results_0160/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-2.1626816 , -3.8276548 , -1.9739904 , ..., -1.6640918 ,
        -3.4563355 , -1.8975035 ],
       [ 0.34442496,  2.2009892 , -4.2959223 , ..., -2.7183883 ,
        18.820694  , -4.6635566 ],
       [ 1.2493331 ,  2.784745  , -3.168465  , ..., -2.0739937 ,
        12.933437  , -0.4737277 ],
       ...,
       [-4.99351   , -1.7031903 ,  3.2899785 , ...,  1.7274523 ,
        -3.1967313 , -7.434282  ],
       [-1.3106475 , 13.305642  , -1.099139  , ...,  0.67260396,
         0.35561472, -3.1092904 ],
       [-1.8581848 ,  1.4106715 , -0.64479494, ..., 12.567974  ,
        -2.0541995 , -3.0111392 ]], dtype=float32)

In [10]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/vgg_15_BN_64'
path = '/home/ec2-user/SageMaker/experiments/vgg_15_BN_64'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)